In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from datetime import datetime
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go
init_notebook_mode(connected = True)

In [153]:
# Достаём данные
df = pd.read_csv("./ms_bucket.csv", index_col=['timestamp'], parse_dates=['timestamp'])

In [154]:
# Избавляемся от пустых строк
df = df.dropna(how='all')

In [24]:
inf = df['Inf']
df = df.drop('Inf', axis=1)

# Список длительностей запросов
duration = [int(item) for item in list(df.columns)]
df.columns = duration

# Сделаем так, чтобы каждый запрос учитывался только один раз
duration_sorted = sorted(duration)
for drt in reversed(range(len(duration_sorted))):
    if drt > 0:
        df[duration_sorted[drt]] = df[duration_sorted[drt]] - df[duration_sorted[drt-1]]

In [25]:
df.head()

,1,10,100,1000,10000,100000,110000,120000,2,20,...,40000,5,50,500,5000,50000,60000,70000,80000,90000
timestamp,,,,,,,,,,,,,,,,,,,,,
1564650000,56905292.0,3390672.0,990153.0,4975.0,151.0,0.0,0.0,0.0,5719411.0,9757800.0,...,7.0,577277.0,642549.0,7133.0,68.0,0.0,0.0,0.0,0.0,0.0
1564650300,56919502.0,3391311.0,990452.0,4976.0,151.0,0.0,0.0,0.0,5720076.0,9759436.0,...,7.0,577426.0,642720.0,7134.0,68.0,0.0,0.0,0.0,0.0,0.0
1564650600,56930005.0,3391812.0,990643.0,4976.0,151.0,0.0,0.0,0.0,5720732.0,9760807.0,...,7.0,577566.0,642862.0,7134.0,68.0,0.0,0.0,0.0,0.0,0.0
1564650900,56940760.0,3392263.0,990810.0,4978.0,151.0,0.0,0.0,0.0,5721240.0,9762006.0,...,7.0,577644.0,642972.0,7139.0,68.0,0.0,0.0,0.0,0.0,0.0
1564651200,56942467.0,3392580.0,991005.0,4978.0,151.0,0.0,0.0,0.0,5721831.0,9762897.0,...,7.0,577739.0,643105.0,7140.0,68.0,0.0,0.0,0.0,0.0,0.0


In [26]:
# Так как будем считать среднее и дисперсию для каждого времени, то транспонируем датасет, чтобы делать расчёты по столбцам
df_t = df.transpose()
df_t.head()

timestamp,1564650000,1564650300,1564650600,1564650900,1564651200,1564651500,1564651800,1564652100,1564652400,1564652700,...,1565424900,1565425200,1565425500,1565425800,1565426100,1565426400,1565426700,1565427000,1565427300,1565427600
1,56905292.0,56919502.0,56930005.0,56940760.0,56942467.0,56943064.0,56944312.0,56944881.0,56945255.0,56946134.0,...,103196825.0,103202578.0,103210180.0,103215518.0,103223026.0,103228905.0,103232285.0,103236181.0,103242723.0,103247790.0
10,3390672.0,3391311.0,3391812.0,3392263.0,3392580.0,3392856.0,3393270.0,3393681.0,3394039.0,3394428.0,...,6417327.0,6418150.0,6419001.0,6419733.0,6420683.0,6421584.0,6421983.0,6422402.0,6422904.0,6423364.0
100,990153.0,990452.0,990643.0,990810.0,991005.0,991217.0,991458.0,991686.0,991908.0,992127.0,...,2080304.0,2080513.0,2080713.0,2080973.0,2081183.0,2081386.0,2081491.0,2081603.0,2081720.0,2081856.0
1000,4975.0,4976.0,4976.0,4978.0,4978.0,4978.0,4978.0,4979.0,4979.0,4981.0,...,12304.0,12304.0,12306.0,12308.0,12308.0,12309.0,12309.0,12309.0,12309.0,12309.0
10000,151.0,151.0,151.0,151.0,151.0,151.0,151.0,151.0,151.0,152.0,...,305.0,305.0,305.0,305.0,305.0,305.0,305.0,305.0,305.0,305.0


In [28]:
# Отсортируем строки по времени обработки запросов 
df_t = df_t.sort_index()
df_t.head()

timestamp,1564650000,1564650300,1564650600,1564650900,1564651200,1564651500,1564651800,1564652100,1564652400,1564652700,...,1565424900,1565425200,1565425500,1565425800,1565426100,1565426400,1565426700,1565427000,1565427300,1565427600
1,56905292.0,56919502.0,56930005.0,56940760.0,56942467.0,56943064.0,56944312.0,56944881.0,56945255.0,56946134.0,...,103196825.0,103202578.0,103210180.0,103215518.0,103223026.0,103228905.0,103232285.0,103236181.0,103242723.0,103247790.0
2,5719411.0,5720076.0,5720732.0,5721240.0,5721831.0,5722398.0,5723205.0,5723886.0,5724227.0,5724791.0,...,11611272.0,11613456.0,11615801.0,11617977.0,11620487.0,11623814.0,11625337.0,11626591.0,11628355.0,11630073.0
3,4739667.0,4741412.0,4742359.0,4743507.0,4744800.0,4746038.0,4747313.0,4749172.0,4750949.0,4752657.0,...,9654693.0,9657509.0,9660812.0,9664526.0,9667821.0,9669704.0,9671693.0,9674990.0,9677742.0,9679521.0
4,1292575.0,1293154.0,1293510.0,1293878.0,1294252.0,1294668.0,1295111.0,1295625.0,1296184.0,1296714.0,...,2752726.0,2753134.0,2753744.0,2754439.0,2755135.0,2755713.0,2756229.0,2756792.0,2757390.0,2757941.0
5,577277.0,577426.0,577566.0,577644.0,577739.0,577841.0,577950.0,578068.0,578195.0,578329.0,...,1180181.0,1180331.0,1180544.0,1180723.0,1180938.0,1181131.0,1181233.0,1181325.0,1181468.0,1181558.0


In [149]:
from statistics import mean

# Сумма последовательности целых чисел от m невключительно до n включительно
def sum_sequence(m, n):
    s = 0
    for i in range(int(m+1), int(n+1)):
        s += i
    return s

# Расчет среднего времени и дисперсии для запросов, длительность обработки которых находится в промежутке 
# от времени time_l до time_r. 
# amt- число запросов с такой длительностью
def calc_mean_duration(drt_l, drt_r, amt):
    mean_drt = amt * sum_sequence(drt_l, drt_r) / (drt_r-drt_l)
    return mean_drt

# Найдем среднее для одного момента времени
def calc_mean_dtm(series):
    duration = series.index
    mean_drt = []
    for idx, drt in enumerate(duration):
        if drt <= 5:
            mean_drt.append(series[drt])
        else:
            mean_drt.append(calc_mean_duration(duration[idx-1], drt, series[drt]))
    mean_dtm = sum(mean_drt)/len(mean_drt)
    return(mean_dtm)

# Найдем дисперсию для одного момента времени
def calc_var_dtm(series, mean):
    duration = series.index
    val_dtm = []
    for idx, drt in enumerate(duration):
        if drt <= 5:
            val_dtm.append(series[drt])
        else:
            val_drt = np.linspace(duration[idx-1]+1, drt, drt-duration[idx-1]) * series[drt] / (drt-duration[idx-1])
            val_dtm = val_dtm + list(val_drt)
    var = sum((np.array(val_dtm)-mean)**2) / len(val_dtm) 
    return var

In [105]:
# Считаем среднее и дисперсию по каждому столбцу
mean_list = []
var_list = []
for colomn_nm in df_t.columns:
    mean_dtm = calc_mean_dtm(df_t[colomn_nm])
    mean_list.append(mean_dtm)
    var = calc_var_dtm(df_t[colomn_nm], mean_dtm)
    var_list.append(var)

In [107]:
df['mean'] = mean_list
df['variation'] = var_list

In [109]:
df.head()

,1,10,100,1000,10000,100000,110000,120000,2,20,...,50,500,5000,50000,60000,70000,80000,90000,mean,variation
timestamp,,,,,,,,,,,,,,,,,,,,,
1564650000,56905292.0,3390672.0,990153.0,4975.0,151.0,0.0,0.0,0.0,5719411.0,9757800.0,...,642549.0,7133.0,68.0,0.0,0.0,0.0,0.0,0.0,2.163569e+06,4.717615e+12
1564650300,56919502.0,3391311.0,990452.0,4976.0,151.0,0.0,0.0,0.0,5720076.0,9759436.0,...,642720.0,7134.0,68.0,0.0,0.0,0.0,0.0,0.0,2.164112e+06,4.719976e+12
1564650600,56930005.0,3391812.0,990643.0,4976.0,151.0,0.0,0.0,0.0,5720732.0,9760807.0,...,642862.0,7134.0,68.0,0.0,0.0,0.0,0.0,0.0,2.164505e+06,4.721693e+12
1564650900,56940760.0,3392263.0,990810.0,4978.0,151.0,0.0,0.0,0.0,5721240.0,9762006.0,...,642972.0,7139.0,68.0,0.0,0.0,0.0,0.0,0.0,2.164907e+06,4.723444e+12
1564651200,56942467.0,3392580.0,991005.0,4978.0,151.0,0.0,0.0,0.0,5721831.0,9762897.0,...,643105.0,7140.0,68.0,0.0,0.0,0.0,0.0,0.0,2.165034e+06,4.723998e+12
